In [ ]:
pip install PySysrev

In [ ]:
import PySysrev, spacy, random
from spacy import displacy

Get spacy ready annotations from Gene Hunter as list of annotated pharagraphs:

In [ ]:
processed_output = PySysrev.processAnnotations(project_id=3144,label='GENE')


Create a model with spacy and name it 'Gene':

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.meta['name'] = 'Gene'

Create NER and an optimizer for training:

In [ ]:
ner = nlp.create_pipe('ner')
ner.add_label('GENE')
nlp.add_pipe(ner)
optimizer = nlp.begin_training()

Training:

In [ ]:
cycles = 30
for i in range(cycles):
    random.shuffle(processed_output)                     #shuffle examples 
    text = [item[0] for item in processed_output]        #get training text items
    annotations = [item[1] for item in processed_output] #get training annotations
    nlp.update(text, annotations, sgd=optimizer, drop=0.6)

In [ ]:
doc = nlp("""Two main classes have been described of lifespan-extension mutants
        in Caenorhabditis elegans. The first consists of genes with activity
        in the mitochondrial electron transport chain, such as clk-1 and isp-1, 
        whose mutation moderately reduces oxidative phosphorylation capacity and prolongs
        life in worms; these mutations established the first link between energy metabolism
        and longevity. The second mutant class is related to hormone mechanisms of the
        insulin/IGF-I signaling (IIS) pathway, such as daf-2 and age-1 mutants,
        which extend lifespan in worms, flies and mice.""")

displacy.serve(doc, style="ent")


Two main classes have been described of lifespan-extension mutants
in Caenorhabditis elegans. The first consists of genes with activity
in the mitochondrial electron transport chain, such as clk-1 GENE and isp-1 GENE, 
whose mutation moderately reduces oxidative phosphorylation capacity and prolongs
life in worms; these mutations established the first link between energy metabolism
and longevity. The second mutant class is related to hormone mechanisms of the
insulin/IGF-I signaling (IIS) pathway, such as daf-2 GENE and age-1 GENE mutants,
which extend lifespan in worms, flies and mice.""")

In [1]:
print("hello")

hello
